In [1]:
# 自动并行
''' 
深度学习框架（例如，MxNet和PyTorch）会在后端⾃动构建计算图。利⽤计算图，系统可以了解所有依赖关
系，并且可以选择性地并⾏执⾏多个不相互依赖的任务以提⾼速度。例如，12.2节中的 图12.2.2独⽴初始化
两个变量。因此，系统可以选择并⾏执⾏它们。
通常情况下单个操作符将使⽤所有CPU或单个GPU上的所有计算资源。例如，即使在⼀台机器上有多个CPU处
理器，dot 操作符也将使⽤所有CPU上的所有核⼼（和线程）。这样的⾏为同样适⽤于单个GPU。因此，并
⾏化对于单设备计算机来说并不是很有⽤，⽽并⾏化对于多个设备就很重要了。虽然并⾏化通常应⽤在多
个GPU之间，但增加本地CPU以后还将提⾼少许性能。例如，[Hadjis et al., 2016]则把结合GPU和CPU的训练
应⽤到计算机视觉模型中。借助⾃动并⾏化框架的便利性，我们可以依靠⼏⾏Python代码实现相同的⽬标。
更⼴泛地考虑，我们对⾃动并⾏计算的讨论主要集中在使⽤CPU和GPU的并⾏计算上，以及计算和通信的并
⾏化内容。
请注意，我们⾄少需要两个GPU来运⾏本节中的实验
'''
import torch
from d2l import torch as d2l

In [2]:
# 基于GPU的并行计算
''' 
让我们从定义⼀个具有参考性的⽤于测试的⼯作负载开始：下⾯的run函数将执⾏10 次“矩阵－矩阵”乘法
时需要使⽤的数据分配到两个变量（x_gpu1和x_gpu2）中，这两个变量分别位于我们选择的不同设备上
------------------------------------------------------------------------------------
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]
x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])
------------------------------------------------------------------------------------
现在我们使⽤函数来数据。我们通过在测量之前预热设备（对设备执⾏⼀次传递）来确保缓存的作⽤不影响
最终的结果。torch.cuda.synchronize()函数将会等待⼀个CUDA设备上的所有流中的所有核⼼的计算
完成。函数接受⼀个device参数，代表是哪个设备需要同步。如果device参数是None（默认值），它将使
⽤current_device()找出的当前设备
------------------------------------------------------------------------------------
run(x_gpu1)
run(x_gpu2)
# 预热设备
torch.cuda.synchronize(devices[0]) # 第一次同步,防止前面的代码中有未同步还在GPU端运行的指令
torch.cuda.synchronize(devices[1])
with d2l.Benchmark('GPU1 time'): # 基准
    run(x_gpu1)
    torch.cuda.synchronize(devices[0]) # 第二次同步,等待所有线程执行完毕后再统计时间
with d2l.Benchmark('GPU2 time'):
    run(x_gpu2)
    torch.cuda.synchronize(devices[1])
------------------------------------------------------------------------------------
如果我们删除两个任务之间的synchronize语句，系统就可以在两个设备上⾃动实现并⾏计算。
------------------------------------------------------------------------------------
with d2l.Benchmark('GPU1 & GPU2'):
    run(x_gpu1)
    run(x_gpu2)
    torch.cuda.synchronize()
------------------------------------------------------------------------------------
在上述情况下，总执⾏时间⼩于两个部分执⾏时间的总和，因为深度学习框架⾃动调度两个GPU设备上的计
算，⽽不需要⽤⼾编写复杂的代码

'''

" \n让我们从定义⼀个具有参考性的⽤于测试的⼯作负载开始：下⾯的run函数将执⾏10 次“矩阵－矩阵”乘法\n时需要使⽤的数据分配到两个变量（x_gpu1和x_gpu2）中，这两个变量分别位于我们选择的不同设备上\n------------------------------------------------------------------------------------\ndevices = d2l.try_all_gpus()\ndef run(x):\n    return [x.mm(x) for _ in range(50)]\nx_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])\nx_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])\n------------------------------------------------------------------------------------\n现在我们使⽤函数来数据。我们通过在测量之前预热设备（对设备执⾏⼀次传递）来确保缓存的作⽤不影响\n最终的结果。torch.cuda.synchronize()函数将会等待⼀个CUDA设备上的所有流中的所有核⼼的计算\n完成。函数接受⼀个device参数，代表是哪个设备需要同步。如果device参数是None（默认值），它将使\n⽤current_device()找出的当前设备\n------------------------------------------------------------------------------------\nrun(x_gpu1)\nrun(x_gpu2)\n# 预热设备\ntorch.cuda.synchronize(devices[0]) # 第一次同步,防止前面的代码中有未同步还在GPU端运行的指令\ntorch.cuda.synchronize(devices[1])\nwith d2l.Benchmark('GPU1 time'): # 基准\n    run(x_gpu1)\n    torch.cuda.synchronize(devices[0]) # 第二次同步

In [6]:
# 并行计算与通信
''' 
在许多情况下，我们需要在不同的设备之间移动数据，⽐如在CPU和GPU之间，或者在不同的GPU之间。例
如，当我们打算执⾏分布式优化时，就需要移动数据来聚合多个加速卡上的梯度。让我们通过在GPU上计算，
然后将结果复制回CPU来模拟这个过程。
'''
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)] # 循环50次矩阵相乘
x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])
# x_gpu2 = torch.rand(size=(4000, 4000), device=devices[1])

# non_blocking(非阻塞) 默认值为False
def copy_to_cpu(x, non_blocking=False): # 复制回cpu
    return [y.to('cpu', non_blocking=non_blocking) for y in x] 
with d2l.Benchmark('在GPU1上运⾏'):
    y = run(x_gpu1) # 在GPU上计算y
    torch.cuda.synchronize()
with d2l.Benchmark('复制到CPU'):
    y_cpu = copy_to_cpu(y) # 将y 复制到cpu
    torch.cuda.synchronize()

在GPU1上运⾏: 1.3329 sec
复制到CPU: 3.3681 sec


In [5]:
''' 
这种⽅式效率不⾼。注意到当列表中的其余部分还在计算时，我们可能就已经开始将y的部分复制
到CPU了。例如，当我们计算⼀个⼩批量的（反传）梯度时。某些参数的梯度将⽐其他参数的梯度更早可
⽤。因此，在GPU仍在运⾏时就开始使⽤PCI-Express总线带宽来移动数据对我们是有利的。在PyTorch中，
to()和copy_()等函数都允许显式的non_blocking参数，这允许在不需要同步时调⽤⽅可以绕过同步。
设置non_blocking=True让我们模拟这个场景。
------------------------------------------------------------------------------------
with d2l.Benchmark('在GPU1上运⾏并复制到CPU'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y, True)
    torch.cuda.synchronize()
'''


" \n这种⽅式效率不⾼。注意到当列表中的其余部分还在计算时，我们可能就已经开始将y的部分复制\n到CPU了。例如，当我们计算⼀个⼩批量的（反传）梯度时。某些参数的梯度将⽐其他参数的梯度更早可\n⽤。因此，在GPU仍在运⾏时就开始使⽤PCI-Express总线带宽来移动数据对我们是有利的。在PyTorch中，\nto()和copy_()等函数都允许显式的non_blocking参数，这允许在不需要同步时调⽤⽅可以绕过同步。\n设置non_blocking=True让我们模拟这个场景。\n------------------------------------------------------------------------------------\nwith d2l.Benchmark('在GPU1上运⾏并复制到CPU'):\n    y = run(x_gpu1)\n    y_cpu = copy_to_cpu(y, True)\n    torch.cuda.synchronize()\n"

In [ ]:
''' 
⼩结
• 现代系统拥有多种设备，如多个GPU和多个CPU，还可以并⾏地、异步地使⽤它们。
• 现代系统还拥有各种通信资源，如PCI Express、存储（通常是固态硬盘或⽹络存储）和⽹络带宽，为
了达到最⾼效率可以并⾏使⽤它们。
• 后端可以通过⾃动化地并⾏计算和通信来提⾼性能。
'''